In [1]:
from cmless.models import Collection, Exhibit
from cmless.parse import parse_cmless, parse_collections
from rich import print
import json
from os import listdir, path


In [2]:
# c = parse_cmless('app/views/special_collections/zoom.md')
# parse_cmless('app/views/special_collections/radio-kdna.md')
# parse_cmless('app/views/special_collections/rock-and-roll.md')
# c = Collection(Title='Test', Slug='asdf', Summary='asdf')

In [2]:
collections = parse_collections('app/views/special_collections')
len(collections)
print(collections[-1])

Collection(
    title='Woman Series',
    slug='woman-series',
    summary="The <em>Woman </em> Series Collection consists of 197 episodes of the <em>Woman</em> series produced 
by WNED in Buffalo, New York from 1972-1977. <em>Woman</em> was a half-hour public affairs talk show covering 
issues of interest to women. The first season, which aired locally, was hosted by Samantha Dean. From 1973 - 1977, 
PBS syndicated the series, and producer Sandra Elkin became the show's host. Each episode focused on a single topic
with featured guest discussants. Guests included well-known and lesser-known activists, journalists, writers, 
scholars, lawyers, artists, psychologists, doctors and others such as Gloria Steinem, editor and co-founder of Ms. 
Magazine; Dorothy Pitman Hughes, African American activist and co-founder of Ms. Magazine; Betty Friedan, author of
*The Feminine Mystique*; Florynce Kennedy, founder of the National Feminist Party; Margaret Sloan, co-founder of 
the National Black Feminist Organization; Karen DeCrow, then president of the National Organization for Women; 
Margaret Dunkle of the Project on the Status of Women for the Association of American Colleges; and Ruth Miller of 
the Buffalo Planned Parenthood Association. Topics discussed on the show included women in sports, the Equal Rights
Amendment, sexuality, marriage, women's health, divorce, the Women's Liberation Movement, motherhood, and ageism, 
among others.",
    resources="- [WNED Buffalo website]( https://www.wned.org/)\n- [Feminist Theory Archive at Brown University]( 
https://www.brown.edu/research/pembroke-center/archives/feminist-theory-archive)\n- [The Schlesinger Library at 
Harvard University's Radcliffe Institute]( https://www.radcliffe.harvard.edu/schlesinger-library)\n- [Sophia Smith 
Collection of Women's History at Smith College]( 
https://www.smith.edu/libraries/research-tools/smith-digital-collections/sophia-smith-collection-womens-history)",
    background="The <em>Woman</em> Series was contributed to the American Archive of Public Broadcasting (AAPB) by 
WNED during the first major AAPB digitization project funded by the Corporation for Public Broadcasting in 2012. 
Once digitized, the show's producer and host Sandra Elkin assisted AAPB archivists in cataloging the digitized 
collection.",
    featured='[![Betty 
Friedan](https://s3.amazonaws.com/americanarchive.org/special-collections/cpb-aacip_81-9995xhm0.jpg)](/catalog/cpb-
aacip_81-9995xhm0)\n[![Gloria 
Steinem](https://s3.amazonaws.com/americanarchive.org/special-collections/cpb-aacip_81-57np5qgv.jpg)](/catalog/cpb-
aacip_81-57np5qgv)\n[![Dorothy Pitman 
Hughes](https://s3.amazonaws.com/americanarchive.org/special-collections/cpb-aacip_81-59c5b5nr.jpg)](/catalog/cpb-a
acip_81-59c5b5nr)\n[![Nora 
Ephron](https://s3.amazonaws.com/americanarchive.org/special-collections/cpb-aacip_81-988gttr0.jpg)](/catalog/cpb-a
acip_81-988gttr0)\n[![Marcia Ann 
Gillespie](https://s3.amazonaws.com/americanarchive.org/special-collections/cpb-aacip_81-69z08t6x.jpg)](/catalog/cp
b-aacip_81-69z08t6x)\n[![Connie Uri and Marie 
Sanchez](https://s3.amazonaws.com/americanarchive.org/special-collections/cpb-aacip_81-67wm3fxh.jpg)](/catalog/cpb-
aacip_81-67wm3fxh)',
    funders=None,
    help='Research is ongoing to determine whether the 197 episodes preserved in the AAPB represent the complete 
<em>Woman</em> series. Records are described at an item level and all records contain brief descriptions and 
subject terms. Search by keyword or individual, or browse all episodes by clicking “Explore the Collection” below 
the search box.\n\nA few of the recordings in the collection consist of segments from multiple programs, and one 
recording in the collection is a 1976 proposal video for the next season. Two of the episodes in the collection are
not available in the Online Reading Room and are only available for research on location at WGBH and the Library of
Congress.',
    terms=None,
    timeline=None,
    sort='as

In [3]:

exhibits = {}

# Parse all the exhibits in the app/views/exhibits directory
for exhibit in listdir('app/views/exhibits'):
    # print(exhibit)
    exhibit_path = path.join('app/views/exhibits', exhibit)
    slug = exhibit.replace('.md', '')
    if exhibits.get(slug) is None:
        exhibits[slug] = []
    if exhibit.endswith('.md'):
        exhibit = parse_cmless(exhibit_path)
        exhibit = Exhibit(**exhibit)
        exhibits[slug].append(exhibit)
    if path.isdir(exhibit_path):
        exhibits[slug] = []
        for exhibit_file in listdir(exhibit_path):
            if exhibit_file.endswith('.md'):
                exhibit = parse_cmless(path.join(exhibit_path, exhibit_file))
                page_slug = exhibit_file.replace('.md', '')
                exhibit = Exhibit(**exhibit,)
                exhibits[slug].append(exhibit)

In [4]:
# print(exhibits)
len(exhibits)
# len(pages)
# pages[0]
# p=exhibits['peabody']
# print(p[0].page)
z = exhibits['zoom']
[ p.page for p in z ]

[3, 5, 2, 4, 1]

In [5]:
with open('collections.json', 'w') as f:
    json.dump([p.model_dump(exclude_none=True, by_alias=True) for p in collections], f, indent=2)
with open('exhibits.json', 'w') as f:
    json.dump(exhibits, f, indent=2, default=lambda x: x.model_dump(exclude_none=True, by_alias=True))

In [ ]:
import re

def cmless2(file):
    with open(file, 'r') as f:
        md = f.read()
    title_pattern = r"^#\s+(.*)"
    section_pattern = r"^##\s+(.*)"
    titles = re.split(title_pattern, md)
    assert titles[0].strip() == '', "There should be nothing before the first title"
    assert len(titles) == 3, "There should be exactly 3 parts to this match"
    title = titles[1].strip()
    body = titles[2].strip()

    
    sections = re.split(section_pattern, body, flags=re.MULTILINE)
    try:
        # If the first section is a number, it means we have a page number
        page = int(sections[0].strip())
    except ValueError:
        # Nevermind
        page = None
        assert sections[0].strip() == '', "First section should be empty if there's no page number"
            
    # Remove the empty first section
    sections = sections[1:]

    # Turn the remaining sections into a dictionary
    sections = {sections[i].strip(): sections[i + 1].strip() for i in range(0, len(sections), 2)}


    sections = {k: v for k, v in sections.items() if k and v}

    result = {
        'Title': title,
        # 'body': body,
        **sections,
    }
    if page is not None:
        result['Page'] = page
    return result

cmless2('app/views/exhibits/zoom.md')